### The with Statement
One of the most common ways to manage resources in Python is to make sure the files we use in our scripts are properly closed after use.

We already explored this concept when we used the with statement when operating on files. The with statement is the most common and pythonic way of invoking context managers in python.

But, what exactly does this have to do with resource management? In order to answer this question, we need to take a peek behind the curtain and examine what our code looks like without a with statement. Here is what the same code would look like without the use of a context manager like with:
```
file = open("file_name.txt", "w")
try:
   file.write("How you gonna win when you ain't right within?")
finally:
   file.close()
```
The alternative to using with would require us to manually open (using open()) and close (using close()) the file we are working on. By using the with statement in the first example, it serves as a context manager where files are automatically closed after script completion and we don’t ever have to worry about the possibility of forgetting to close a resource. Remember, leaving our resources open will hog up our finite computer resources.

### Class Based Context Managers
Now that we have an understanding of why we need context managers and the power of the with statement, it is essential for us to know what’s happening under the hood to gain a much deeper understanding of the concept. The best way to see the internal workings of a context manager (such as the with statement) is by creating our own!

One of the two approaches of creating context managers is referred to as the class-based approach. The class-based approach of writing context managers requires explicitly defining and implementing the following two methods inside of a class:

1. An __enter__ method
The __enter__ method allows for the setup of context managers. This method commonly takes care of opening resources (like files). This method also begins what is known as the runtime context - the period of time in which a script runs. In our previous examples, it was the time in which the code passed into the with statement code block was executed (basically everything under the with statement).

2. An __exit__ method
The __exit__ ensures the breakdown of the context manager. This method commonly takes care of closing open resources that are no longer in use.
To visualize these methods and the approach, let’s take a look at a custom class-based context manager below:
```
class ContextManager:
  def __init__(self):
    print('Initializing class...')
 
  def __enter__(self):
    print('Entering context...')
 
  def __exit__(self, *exc):
    print('Exiting context...')
```
Here, we defined a new class called ContextManager (to be extra explicit) and implemented the required methods. By defining these two methods, we are implementing the context management protocol - a guideline for the required methods for a context manager. Don’t get too caught up in the arguments passed to each method, we will talk through them in the next exercises, but they are required to not experience an error.

Implementing the context management protocol allows us to immediately invoke the class using the with statement as shown below:
```
with ContextManager() as cm:
  print('Code inside with statement')
```
Here we invoke the ContextManager class with a with statement.
After running the code, our output of this context manager would be:
```
Initializing class...
Entering context...
Code inside with statement
Exiting context...
```
The above shows that our context manager class is executed in the following sequence:

__init__ method
__enter__ method
The code in the with statement block
__exit__ method
Let’s practice getting down the basics of writing a class-based context manager in addition to the execution flow before diving deeper into the __enter__ and __exit__ methods.

Note: in reality, we wouldn’t have to create a context manager that opens a file because there’s already an open() built-in function that you can run with a with statement that will open and close a file. However, open() has its limitations, and knowing this base structure will allow us to create our own custom and more advanced context managers that can do much more than open()!

In [ ]:
class PoemFiles:
  def __init__(self, poem_file, mode):
    print('Starting up a poem context manager')
    self.file = poem_file
    self.mode = mode

  def __enter__(self):
    print('Opening poem file')
    self.opened_poem_file = open(self.file, self.mode)
    return self.opened_poem_file

  def __exit__(self, *exc):
    print('Closing poem file')
    self.opened_poem_file.close()


### Handling Exceptions I
Remember this?

`def __exit__(self, *exc):`

It’s time to address the big mystery. What in the world is the *exc parameter in the __exit__ method we have been writing so far?

Well, context managers play an important role in handling exceptions. Recall exceptions are errors that happen within the runtime of a code, terminating it before its completion. Within a context manager, the __exit__ method is responsible for dealing with any exceptions. It can implement how to close the file and any other operations we want to perform if an exception occurs.

So far, we have been using *exc to fill in the argument requirements for our context managers __exit__ method. If we went back and wrote this instead:
```
def __exit__(self):

We would have been met with a puzzling error:

__exit__() takes 1 positional argument but 4 were given.
```
This is because the __exit__ method needs four total arguments! In the past exercises, we ignored this requirement by using the * operator to tell the method we will pass a variable number of arguments even though we never did. It was a good way to put the above error on hold, but now let’s dive into what these required arguments are and how to use them so that we can master the __exit__ method.

The __exit__ method has three required arguments (in addition to self):
1. An exception type: which indicates the class of exception (i.e. AttributeError class, or NameError class)
2. An exception value: the actual value of the error
3. A traceback: a report detailing the sequence of steps that caused the error and all the details needed to fix the error.

Let’s take a look at an example context manager that deals with exceptions in its __exit__ method:
```
class OpenFile:
 
 def __init__(self, file, mode):
   self.file = file
   self.mode = mode

 def __enter__(self):
   self.opened_file = open(self.file, self.mode)
   return self.opened_file
 
 def __exit__(self, exc_type, exc_val, traceback):
   print(exc_type)
   print(exc_val)
   print(traceback)
   self.opened_file.close()
```
Note that exc_type, exc_value, and traceback are completely arbitrary names. We can use any name we want for these parameters as long as it does not hinder the readability of our code. In general, it’s best practice to be as descriptive as possible!